# Coursera dataset

In [ ]:
#dataset preparation

import pandas as pd
from collections import defaultdict

df = pd.read_csv("big_student_mooc_24.csv") 

# Filter by 'viewed' == 1
df = df[df["viewed"] == 1]

#Encode course and user as numeric IDs
df["course_num"], course_lookup = pd.factorize(df["course_id"])
df["user_num"], user_lookup = pd.factorize(df["userid_DI"])

#Group by user to create PAAC lines
user_course_dict = defaultdict(list)
for row in df.itertuples():
    user_course_dict[row.user_num].append(row.course_num)

#  Write to PAAC format file
with open("mooc_paac.txt", "w") as f:
    for user_id, course_ids in user_course_dict.items():
        course_str = " ".join(map(str, sorted(set(course_ids))))
        f.write(f"{user_id} 0 {course_str}\n")


In [20]:
df1 = df[df["course_id"]== "PH207x"]

In [21]:
df1

,Id,course_id,userid_DI,viewed,certified,user_num,course_num
263,611,PH207x,MHxPC130564939,1,1,6,1
348,834,PH207x,MHxPC130439995,1,1,11,1
413,1001,PH207x,MHxPC130559131,1,1,15,1
484,1190,PH207x,MHxPC130438664,1,1,19,1
676,1705,PH207x,MHxPC130240764,1,1,23,1
...,...,...,...,...,...,...,...
58277,151670,PH207x,MHxPC130107258,1,1,3061,1
59013,153418,PH207x,MHxPC130487961,1,1,3079,1
61379,159282,PH207x,MHxPC130268688,1,1,3144,1
63303,164206,PH207x,MHxPC130233256,1,1,3189,1


In [ ]:
# import pandas as pd
# from collections import defaultdict
# from sklearn.model_selection import train_test_split

# # Load dataset
# df = pd.read_csv("big_student_mooc_24.csv")
# df = df[df["viewed"] == 1]

# # Encode user and course IDs
# df["user_num"], user_lookup = pd.factorize(df["userid_DI"])
# df["course_num"], course_lookup = pd.factorize(df["course_id"])

# user_course_dict = defaultdict(list)
# for row in df.itertuples():
#     user_course_dict[row.user_num].append(row.course_num)

# qualified_users = {
#     user: sorted(set(courses))
#     for user, courses in user_course_dict.items()
#     if len(set(courses)) >= 4
# }
# print(len(qualified_users))

# filtered_rows = []
# for user, courses in qualified_users.items():
#     for course in courses:
#         filtered_rows.append((user, course))

# filtered_df = pd.DataFrame(filtered_rows, columns=["user_num", "course_num"])
# print(filtered_df.shape)

# qualified_user_ids = list(qualified_users.keys())
# train_users, test_users = train_test_split(qualified_user_ids, train_size=0.7, random_state=42)

# train_df = filtered_df[filtered_df["user_num"].isin(train_users)]
# test_df = filtered_df[filtered_df["user_num"].isin(test_users)]

# def write_paac_format(df_subset, filename):
#     uc_dict = defaultdict(list)
#     for row in df_subset.itertuples():
#         uc_dict[row.user_num].append(row.course_num)

#     with open(filename, "w") as f:
#         for user_id, course_ids in uc_dict.items():
#             unique_courses = sorted(set(course_ids))
#             course_str = " ".join(map(str, unique_courses))
#             f.write(f"{user_id} {course_str}\n")

# write_paac_format(train_df, "user_to_course_paac_train.txt")
# write_paac_format(test_df, "user_to_course_paac_test.txt")

# pd.Series(user_lookup).to_csv("user_lookup.csv", index_label="user_num", header=["user_id"])
# pd.Series(course_lookup).to_csv("course_lookup.csv", index_label="course_num", header=["course_id"])


1935
(8717, 2)


In [ ]:
import pandas as pd
from collections import defaultdict
import random

random.seed(42)

# Load dataset
df = pd.read_csv("big_student_mooc_24.csv")
df = df[df["viewed"] == 1]

# Encode user and course IDs
df["user_num"], user_lookup = pd.factorize(df["userid_DI"])
df["course_num"], course_lookup = pd.factorize(df["course_id"])

# Build user -> list of courses
user_course_dict = defaultdict(list)
for row in df.itertuples():
    user_course_dict[row.user_num].append(row.course_num)

# Filter users with ≥4 unique courses
qualified_users = {
    user: sorted(set(courses))
    for user, courses in user_course_dict.items()
    if len(set(courses)) >= 4
}
print(f"# qualified users: {len(qualified_users)}")
# print(qualified_users)

# Split each user's items into train/test
train_user_course = defaultdict(list)
test_user_course = defaultdict(list)

for user, courses in qualified_users.items():
    random.shuffle(courses)
    split_point = int(len(courses) * 0.8)  
    train_courses = courses[:split_point]
    test_courses = courses[split_point:]
    
    if len(train_courses) >= 1:
        train_user_course[user] = train_courses
    if len(test_courses) >= 1:
        test_user_course[user] = test_courses

print(train_user_course)
print(test_user_course)

print(f"# users in train_user_course: {len(train_user_course)}")
print(f"# users in test_user_course: {len(test_user_course)}")

#  Write PAAC-style output files
def write_paac_file(mapping, filename):
    with open(filename, "w") as f:
        for user_id, item_ids in mapping.items():
            course_str = " ".join(map(str, sorted(item_ids)))
            f.write(f"{user_id} {course_str}\n")

write_paac_file(train_user_course, "new_user_to_course_paac_train.txt")
write_paac_file(test_user_course, "new_user_to_course_paac_test.txt")

# Save ID mappings
pd.Series(user_lookup).to_csv("user_lookup.csv", index_label="user_num", header=["user_id"])
pd.Series(course_lookup).to_csv("course_lookup.csv", index_label="course_num", header=["course_id"])


# qualified users: 1935
defaultdict(<class 'list'>, {7: [5, 2, 3, 9], 42: [9, 7, 1], 47: [6, 3, 4, 12, 9], 75: [3, 6, 7, 8], 76: [11, 1, 9], 80: [1, 10, 7], 81: [1, 7, 10], 131: [6, 4, 10], 162: [12, 1, 7], 164: [8, 1, 9], 186: [4, 10, 2], 201: [1, 7, 9, 12], 226: [7, 5, 6], 243: [4, 5, 3, 1], 249: [2, 7, 11], 250: [7, 1, 3], 255: [6, 11, 8], 259: [5, 7, 1], 267: [8, 7, 6], 275: [2, 4, 6], 283: [1, 8, 11], 294: [2, 11, 9], 307: [2, 7, 5], 313: [7, 8, 1], 373: [7, 1, 6], 375: [2, 7, 4, 1], 396: [1, 10, 7], 400: [7, 1, 11], 409: [5, 10, 0, 2, 1, 11, 6, 7], 413: [1, 2, 5, 3], 481: [7, 1, 6, 8], 496: [11, 6, 9, 2, 7], 503: [8, 1, 7], 537: [3, 1, 5], 592: [0, 1, 2, 4], 628: [2, 1, 9], 651: [9, 1, 7], 668: [9, 11, 6, 1], 675: [6, 8, 1, 7], 691: [7, 1, 3], 696: [9, 8, 2, 3], 737: [10, 1, 7], 784: [2, 9, 1, 7], 819: [11, 1, 2], 828: [7, 9, 4, 2], 895: [3, 8, 5, 11, 0, 4], 995: [6, 0, 11, 1, 3, 7, 8, 5], 1026: [1, 8, 6], 1061: [2, 3, 4], 1095: [2, 4, 5], 1097: [7, 8, 6], 1122: [7, 9, 3, 8, 1, 2

In [6]:
df = pd.read_csv("big_student_mooc_24.csv")
df = df[df["viewed"] == 1]
print(df[df["userid_DI"]== "MHxPC130378021"])

            Id course_id       userid_DI  viewed  certified
76         193     CB22x  MHxPC130378021       1          0
77         194     CS50x  MHxPC130378021       1          0
176954  319065     6.00x  MHxPC130378021       1          0
176955  319066     7.00x  MHxPC130378021       1          0


In [ ]:
# import pandas as pd
# from collections import defaultdict
# from sklearn.model_selection import train_test_split

# # Load dataset and filter only viewed
# df = pd.read_csv("big_student_mooc_24.csv")
# df = df[df["viewed"] == 1]

# # Encode user and course IDs
# df["user_num"], user_lookup = pd.factorize(df["userid_DI"])
# df["course_num"], course_lookup = pd.factorize(df["course_id"])

# user_course_dict = defaultdict(list)
# for row in df.itertuples():
#     user_course_dict[row.user_num].append(row.course_num)

# qualified_users = {
#     user: sorted(set(courses))
#     for user, courses in user_course_dict.items()
#     if len(set(courses)) >= 4
# }

# filtered_rows = []
# for user, courses in qualified_users.items():
#     for course in courses:
#         filtered_rows.append((user, course))

# filtered_df = pd.DataFrame(filtered_rows, columns=["user_num", "course_num"])


# df_full = pd.read_csv("big_student_mooc_24.csv")

# # Encode course_id consistently
# df_full["course_num"] = df_full["course_id"].map({v: k for k, v in dict(enumerate(course_lookup)).items()})


# course_views = df.groupby("course_num")["user_num"].nunique()
# course_certs = df_full[df_full["certified"] == 1].groupby("course_num")["userid_DI"].nunique()

# completion_df = pd.DataFrame({
#     "views": course_views,
#     "certs": course_certs
# }).fillna(0)
# completion_df["completion_rate"] = completion_df["certs"] / completion_df["views"]

# # ==== Quantile-based Replication Factor Assignment ====
# q_low = completion_df["completion_rate"].quantile(0.33)
# q_high = completion_df["completion_rate"].quantile(0.66)

# def assign_replication(rate):
#     if rate >= q_high:
#         return 3
#     elif rate >= q_low:
#         return 2
#     else:
#         return 1

# completion_df["replication_factor"] = completion_df["completion_rate"].apply(assign_replication)
# replication_map = completion_df["replication_factor"].to_dict()

# replicated_rows = []
# for row in filtered_df.itertuples():
#     factor = replication_map.get(row.course_num, 1)
#     replicated_rows.extend([(row.user_num, row.course_num)] * factor)

# replicated_df = pd.DataFrame(replicated_rows, columns=["user_num", "course_num"])

# print("Original interactions:", len(filtered_df))
# print("Replicated interactions:", len(replicated_df))
# print("Replication factors:\n", completion_df["replication_factor"].value_counts())

# qualified_user_ids = list(qualified_users.keys())
# train_users, test_users = train_test_split(qualified_user_ids, train_size=0.7, random_state=42)

# train_df = replicated_df[replicated_df["user_num"].isin(train_users)]
# test_df = replicated_df[replicated_df["user_num"].isin(test_users)]

# def write_paac_format(df_subset, filename):
#     uc_dict = defaultdict(list)
#     for row in df_subset.itertuples():
#         uc_dict[row.user_num].append(row.course_num)

#     with open(filename, "w") as f:
#         for user_id, course_ids in uc_dict.items():
#             unique_courses = sorted(set(course_ids))
#             course_str = " ".join(map(str, unique_courses))
#             f.write(f"{user_id} {course_str}\n")

# write_paac_format(train_df, "abcd_user_to_course_paac_replicated_train.txt")
# write_paac_format(test_df, "abcd_user_to_course_paac_replicated_test.txt")

# pd.Series(user_lookup).to_csv("user_lookup.csv", index_label="user_num", header=["user_id"])
# pd.Series(course_lookup).to_csv("course_lookup.csv", index_label="course_num", header=["course_id"])


Original interactions: 8717
Replicated interactions: 16350
Replication factors:
 3    5
1    4
2    4
Name: replication_factor, dtype: int64


In [ ]:
import pandas as pd
from collections import defaultdict
import random

random.seed(42)

# Load dataset and filter only viewed
df = pd.read_csv("big_student_mooc_24.csv")
df = df[df["viewed"] == 1]

# Encode user and course IDs
df["user_num"], user_lookup = pd.factorize(df["userid_DI"])
df["course_num"], course_lookup = pd.factorize(df["course_id"])

# Build user -> list of courses
user_course_dict = defaultdict(list)
for row in df.itertuples():
    user_course_dict[row.user_num].append(row.course_num)

# Filter users with ≥4 unique courses
qualified_users = {
    user: sorted(set(courses))
    for user, courses in user_course_dict.items()
    if len(set(courses)) >= 4
}
print(f"# qualified users: {len(qualified_users)}")

# Prepare filtered DataFrame
filtered_rows = []
for user, courses in qualified_users.items():
    for course in courses:
        filtered_rows.append((user, course))

filtered_df = pd.DataFrame(filtered_rows, columns=["user_num", "course_num"])

# ==== NEW STEP: Compute course completion rates d====
# Reload full dataset for certified column
df_full = pd.read_csv("big_student_mooc_24.csv")
df_full["course_num"] = df_full["course_id"].map({v: k for k, v in dict(enumerate(course_lookup)).items()})

# Views and Certifications per course
course_views = df.groupby("course_num")["user_num"].nunique()
course_certs = df_full[df_full["certified"] == 1].groupby("course_num")["userid_DI"].nunique()

completion_df = pd.DataFrame({
    "views": course_views,
    "certs": course_certs
}).fillna(0)
completion_df["completion_rate"] = completion_df["certs"] / completion_df["views"]

q_low = completion_df["completion_rate"].quantile(0.33)
q_high = completion_df["completion_rate"].quantile(0.66)

def assign_replication(rate):
    if rate >= q_high:
        return 3
    elif rate >= q_low:
        return 2
    else:
        return 1

completion_df["replication_factor"] = completion_df["completion_rate"].apply(assign_replication)
replication_map = completion_df["replication_factor"].to_dict()

# Replicate interactions based on completion rate 
replicated_rows = []
for row in filtered_df.itertuples():
    factor = replication_map.get(row.course_num, 1)
    replicated_rows.extend([(row.user_num, row.course_num)] * factor)

replicated_df = pd.DataFrame(replicated_rows, columns=["user_num", "course_num"])
print("Original interactions:", len(filtered_df))
print("Replicated interactions:", len(replicated_df))
print("Replication factors:\n", completion_df["replication_factor"].value_counts())

#Per-user split after replication
user_course_dict = defaultdict(list)
for row in replicated_df.itertuples():
    user_course_dict[row.user_num].append(row.course_num)

train_user_course = defaultdict(list)
test_user_course = defaultdict(list)

for user, courses in user_course_dict.items():
    if len(courses) < 4:
        continue
    random.shuffle(courses)
    split_point = int(len(courses) * 0.8)
    train_courses = courses[:split_point]
    test_courses = courses[split_point:]
    if len(train_courses) >= 1:
        train_user_course[user] = train_courses
    if len(test_courses) >= 1:
        test_user_course[user] = test_courses

print(f"# users in train_user_course: {len(train_user_course)}")
print(f"# users in test_user_course: {len(test_user_course)}")

# Write PAAC format 
def write_paac_file(mapping, filename):
    with open(filename, "w") as f:
        for user_id, item_ids in mapping.items():
            course_str = " ".join(map(str, sorted(item_ids)))
            f.write(f"{user_id} {course_str}\n")

write_paac_file(train_user_course, "certified_bias_user_to_course_train.txt")
write_paac_file(test_user_course, "certified_bias_user_to_course_test.txt")


pd.Series(user_lookup).to_csv("user_lookup.csv", index_label="user_num", header=["user_id"])
pd.Series(course_lookup).to_csv("course_lookup.csv", index_label="course_num", header=["course_id"])


# qualified users: 1935
Original interactions: 8717
Replicated interactions: 16350
Replication factors:
 3    5
1    4
2    4
Name: replication_factor, dtype: int64
# users in train_user_course: 1935
# users in test_user_course: 1935


# netflix -- we tried to extend to another domain, but couldn't get substantial results out of it. But here is the experiment

In [2]:
import pandas as pd

file_path = 'combined_data_1.txt'

with open(file_path, 'r') as f:
    lines = f.readlines()

movie_id = int(lines[0].strip().replace(":", ""))

records = []
for line in lines[1:]:
    parts = line.strip().split(',')
    if len(parts) != 3:
        continue  # skip malformed or empty lines
    customer_id, rating, date = parts
    records.append((movie_id, int(customer_id), int(rating), date))

df = pd.DataFrame(records, columns=['MovieID', 'CustomerID', 'Rating', 'Date'])

print(df.head(20))


    MovieID  CustomerID  Rating        Date
0         1     1488844       3  2005-09-06
1         1      822109       5  2005-05-13
2         1      885013       4  2005-10-19
3         1       30878       4  2005-12-26
4         1      823519       3  2004-05-03
5         1      893988       3  2005-11-17
6         1      124105       4  2004-08-05
7         1     1248029       3  2004-04-22
8         1     1842128       4  2004-05-09
9         1     2238063       3  2005-05-11
10        1     1503895       4  2005-05-19
11        1     2207774       5  2005-06-06
12        1     2590061       3  2004-08-12
13        1        2442       3  2004-04-14
14        1      543865       4  2004-05-28
15        1     1209119       4  2004-03-23
16        1      804919       4  2004-06-10
17        1     1086807       3  2004-12-28
18        1     1711859       4  2005-05-08
19        1      372233       5  2005-11-23


In [19]:
df[df["CustomerID"] == 885013]["MovieID"].unique()

array([1], dtype=int64)

In [20]:
df.shape

(24053764, 4)

In [15]:
import numpy as np

In [18]:
# After reading in a few files, collect CustomerIDs
all_customer_ids = df['CustomerID'].unique()
sample_users = np.random.choice(all_customer_ids, size=50000, replace=False)

df_new = df[df['CustomerID'].isin(sample_users)]


In [19]:
df_new.shape

(2550942, 4)

In [21]:
df_new['Rating'].unique()

array([3, 4, 2, 5, 1], dtype=int64)

In [23]:
df_new.columns

Index(['MovieID', 'CustomerID', 'Rating', 'Date'], dtype='object')

In [22]:
import pandas as pd

# Ensure only required columns and no duplicates
df_cleaned = df[['CustomerID', 'MovieID']].drop_duplicates()

# Group by CustomerID and get list of MovieIDs per user
user_movie_dict = df_cleaned.groupby('CustomerID')['MovieID'].apply(list)

# Convert to list of lines in the required format
paac_lines = []
for user_id, movie_list in user_movie_dict.items():
    sorted_movies = sorted(movie_list)  # sort if desired
    line = f"{user_id} " + " ".join(str(mid) for mid in sorted_movies)
    paac_lines.append(line)


In [25]:
print(paac_lines)

['59 1', '94 1', '283 1', '358 1', '392 1', '424 1', '439 1', '462 1', '592 1', '624 1', '665 1', '735 1', '785 1', '815 1', '844 1', '911 1', '952 1', '1069 1', '1236 1', '1355 1', '1427 1', '1498 1', '1518 1', '1527 1', '1588 1', '1590 1', '1636 1', '1664 1', '1689 1', '1691 1', '1738 1', '1753 1', '1874 1', '1894 1', '1944 1', '1983 1', '2040 1', '2190 1', '2347 1', '2379 1', '2431 1', '2447 1', '2549 1', '2630 1', '2697 1', '2762 1', '2781 1', '2787 1', '2820 1', '2909 1', '3019 1', '3049 1', '3188 1', '3204 1', '3260 1', '3290 1', '3344 1', '3350 1', '3354 1', '3404 1', '3439 1', '3466 1', '3597 1', '3628 1', '3687 1', '3900 1', '3906 1', '3930 1', '3931 1', '3958 1', '4084 1', '4167 1', '4170 1', '4365 1', '4389 1', '4540 1', '4581 1', '4597 1', '4607 1', '4630 1', '4653 1', '4700 1', '4773 1', '4796 1', '4797 1', '4853 1', '4873 1', '4906 1', '4963 1', '5022 1', '5109 1', '5126 1', '5228 1', '5353 1', '5459 1', '5474 1', '5496 1', '5549 1', '5573 1', '5650 1', '5710 1', '5762 1'

In [ ]:
import pandas as pd
import random

# Step 1: Drop duplicates & select relevant columns
df_cleaned = df[['CustomerID', 'MovieID']].drop_duplicates()

# Step 2: Group by user and filter users with ≥4 movies
user_movie_dict = df_cleaned.groupby('CustomerID')['MovieID'].apply(list)
filtered_users = user_movie_dict[user_movie_dict.apply(len) >= 3]

print(f"Total users with ≥4 interactions: {len(filtered_users)}")

# Step 3: Create PAAC-style lines (user_id movie_id movie_id ...)
paac_lines = [
    f"{user_id} " + " ".join(str(mid) for mid in sorted(movie_list))
    for user_id, movie_list in filtered_users.items()
]

print(f"Total PAAC lines: {len(paac_lines)}")

# Step 4: Shuffle and split into train/test (70/30)
random.seed(42)
random.shuffle(paac_lines)

split_index = int(len(paac_lines) * 0.7)
train_lines = paac_lines[:split_index]
test_lines = paac_lines[split_index:]

print(f"Train lines: {len(train_lines)}, Test lines: {len(test_lines)}")

# Step 5: Save to files if data exists
if train_lines and test_lines:
    with open("paac_train.txt", "w") as f:
        f.write("\n".join(train_lines))

    with open("paac_test.txt", "w") as f:
        f.write("\n".join(test_lines))

    print("Files written successfully.")
    print("Sample train line:\n", train_lines[0])
    print("Sample test line:\n", test_lines[0])
else:
    print("Error: Train or test set is empty. Try lowering the interaction threshold or increasing your dataset.")


Total users with ≥4 interactions: 0
Total PAAC lines: 0
Train lines: 0, Test lines: 0
Error: Train or test set is empty. Try lowering the interaction threshold or increasing your dataset.


In [6]:
import pandas as pd
from collections import defaultdict
import random

df_cleaned = df[['CustomerID', 'MovieID']].drop_duplicates()

# Step 2: Group by user and filter users with ≥3 movies
user_movie_dict = defaultdict(list)
for row in df_cleaned.itertuples():
    user_movie_dict[row.CustomerID].append(row.MovieID)

# Only keep users with ≥3 unique movies
qualified_users = {
    user: sorted(set(movies))
    for user, movies in user_movie_dict.items()
    if len(set(movies)) >= 1
}
print(f"✅ Qualified users (≥3 movies): {len(qualified_users)}")

# Step 3: Per-user item split into 80% train, 20% test
train_user_movie = defaultdict(list)
test_user_movie = defaultdict(list)

for user, movies in qualified_users.items():
    random.shuffle(movies)
    split_point = int(len(movies) * 0.8)
    train_items = movies[:split_point]
    test_items = movies[split_point:]
    
    if train_items:
        train_user_movie[user] = train_items
    if test_items:
        test_user_movie[user] = test_items

print(f"Users in train: {len(train_user_movie)}")
print(f"Users in test: {len(test_user_movie)}")

# Step 4: Write PAAC format
def write_paac(mapping, filename):
    with open(filename, "w") as f:
        for user_id, items in mapping.items():
            movie_str = " ".join(map(str, sorted(items)))
            f.write(f"{user_id} {movie_str}\n")

write_paac(train_user_movie, "user_to_movie_paac_train.txt")
write_paac(test_user_movie, "user_to_movie_paac_test.txt")

print("✅ PAAC train/test files written successfully.")


KeyboardInterrupt: 

# Yelp 2018 -- lastly, we tried using replications for yelp 2018 dataset on the basis of rating and time - stamps, but couldn't find substantial results

In [5]:
!pip install "dask[complete]"
# or, more minimally
!pip install dask


In [7]:
import pandas as pd

reader = pd.read_json(
    "yelp_academic_dataset_review.json",
    lines=True,
    chunksize=100_000
)

samples = []
for chunk in reader:
    # pick e.g. 10 random reviews from each 100k‑chunk
    samples.append(chunk.sample(n=10000))
    
df_random = pd.concat(samples, ignore_index=True)


In [8]:
df_random.shape

(700000, 9)

In [9]:
df_random

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,pv4TIgaLDbGSIPMEgXN83g,U1iO4R4H0ZhC-jEAOYNmAw,GBTPC53ZrG1ZBY3DT8Mbcw,5,0,0,0,Restaurant was suggested for their happy hour ...,2017-08-30 12:07:56
1,LJ8LcA9dUBaigVaDzt1Wlg,hvGspChzUx9daHYImMYDxw,68ThEdiK0eWCEgGEmV9Tng,4,0,0,0,"We were so excited to try Southgate, and I'm g...",2016-08-08 02:42:22
2,9LfI40q4gmMMZjFkbf-buA,UNCwaqH-rKsTksW6YHVbYw,M0r9lUn2gLFYgIwIfG8-bQ,4,0,0,0,"Burger was great, the atmosphere was eclectic ...",2014-06-28 20:43:00
3,f-IU2WuzMRkYGp01webwyw,R-W0T3bemRPqWcm2V9oY-w,OyHbS1sJ4CguExlWa-XWDg,4,1,0,0,"We had Shrimp Scampi, fried shrimp, stuffed mu...",2014-09-09 18:56:55
4,sEE51VtGkc1b-zP4pRlwDw,N1Hj5UG05p2hK4pcYOTlbA,Yit7GvwzEYbIzkNhxeqC7w,5,6,0,0,Came here because of reviews were good. But I ...,2016-08-06 13:12:02
...,...,...,...,...,...,...,...,...,...
699995,PI2LjTPoK97KvIejNWTARA,9EN3rJWPiQ4JSq0TyR0jEg,8JEcsyj5jqsSqmb0Xvwnbw,5,0,0,0,I am just in town for vacation and wish that w...,2017-12-28 16:56:46
699996,DrTmn6TfkAn6PwpFiaEzBw,pzq4a3asAB2U1bDugVn6nA,nenqighGaoZ4O-DHx2qmMA,1,2,0,0,Went to this place per my car insurance compan...,2017-08-31 15:33:37
699997,K_yCOahrU0c-Gq-ex9RoqA,tSv-Q6BOkP3UieqW0eGM0w,ZNOc9LErlBDtJF3Y7LQtwA,5,3,0,1,I love this place!! I've been coming to it sin...,2020-03-30 14:43:55
699998,_9Qc7GjRUFTX23sl4Bh7sA,usrsUOC93C4i-yXlS6HcLA,2njTjQ35skXN171KoobeGg,1,2,0,0,I normally love this place especially the scon...,2020-06-03 23:19:26


In [13]:

df = df_random[["business_id", "user_id", "stars", "date"]]


In [14]:
df

,business_id,user_id,stars,date
0,GBTPC53ZrG1ZBY3DT8Mbcw,U1iO4R4H0ZhC-jEAOYNmAw,5,2017-08-30 12:07:56
1,68ThEdiK0eWCEgGEmV9Tng,hvGspChzUx9daHYImMYDxw,4,2016-08-08 02:42:22
2,M0r9lUn2gLFYgIwIfG8-bQ,UNCwaqH-rKsTksW6YHVbYw,4,2014-06-28 20:43:00
3,OyHbS1sJ4CguExlWa-XWDg,R-W0T3bemRPqWcm2V9oY-w,4,2014-09-09 18:56:55
4,Yit7GvwzEYbIzkNhxeqC7w,N1Hj5UG05p2hK4pcYOTlbA,5,2016-08-06 13:12:02
...,...,...,...,...
699995,8JEcsyj5jqsSqmb0Xvwnbw,9EN3rJWPiQ4JSq0TyR0jEg,5,2017-12-28 16:56:46
699996,nenqighGaoZ4O-DHx2qmMA,pzq4a3asAB2U1bDugVn6nA,1,2017-08-31 15:33:37
699997,ZNOc9LErlBDtJF3Y7LQtwA,tSv-Q6BOkP3UieqW0eGM0w,5,2020-03-30 14:43:55
699998,2njTjQ35skXN171KoobeGg,usrsUOC93C4i-yXlS6HcLA,1,2020-06-03 23:19:26


In [16]:
import pandas as pd
from collections import defaultdict
import random

random.seed(42)
df = df.copy()
# Encode user and item (business) IDs
df["item_num"], item_lookup = pd.factorize(df["business_id"])
df["user_num"], user_lookup = pd.factorize(df["user_id"])

# Step 1: Build user -> list of item_nums
user_item_dict = defaultdict(list)
for row in df.itertuples():
    user_item_dict[row.user_num].append(row.item_num)

# Step 2: Filter users with ≥4 unique items
qualified_users = {
    user: sorted(set(items))
    for user, items in user_item_dict.items()
    if len(set(items)) >= 4
}
print(f"# qualified users: {len(qualified_users)}")

# Step 3: Split into train/test
train_user_course = defaultdict(list)
test_user_course = defaultdict(list)

for user, items in qualified_users.items():
    random.shuffle(items)
    split_point = int(len(items) * 0.8)
    train_items = items[:split_point]
    test_items = items[split_point:]
    
    if train_items:
        train_user_course[user] = train_items
    if test_items:
        test_user_course[user] = test_items

print(f"# users in train_user_course: {len(train_user_course)}")
print(f"# users in test_user_course: {len(test_user_course)}")

# Step 4: Write PAAC-style files
def write_paac_file(mapping, filename):
    with open(filename, "w") as f:
        for user_id, item_ids in mapping.items():
            line = f"{user_id} " + " ".join(map(str, sorted(item_ids)))
            f.write(line + "\n")

write_paac_file(train_user_course, "new_user_to_item_paac_train.txt")
write_paac_file(test_user_course, "new_user_to_item_paac_test.txt")

# Step 5: Save ID mappings
pd.Series(user_lookup).to_csv("yelp_user_lookup.csv", index_label="user_num", header=["user_id"])
pd.Series(item_lookup).to_csv("yelp_item_lookup.csv", index_label="item_num", header=["item_id"])


# qualified users: 25766
# users in train_user_course: 25766
# users in test_user_course: 25766


In [21]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
import random

random.seed(42)

# Copy to avoid SettingWithCopyWarning
df = df.copy()

# Encode user and item (business) IDs
df["item_num"], item_lookup = pd.factorize(df["business_id"])
df["user_num"], user_lookup = pd.factorize(df["user_id"])

# Step 1: Build user -> list of (item_num, stars)
user_item_dict = defaultdict(list)
for row in df.itertuples():
    user_item_dict[row.user_num].append((row.item_num, row.stars))

# Step 2: Filter users with ≥4 unique items
qualified_users = {}
for user, items in user_item_dict.items():
    unique_items = set(item for item, _ in items)
    if len(unique_items) >= 4:
        qualified_users[user] = items

print(f"# qualified users: {len(qualified_users)}")

# Step 3: Flatten interactions (user, item, stars) only for qualified users
filtered_rows = []
for user, items in qualified_users.items():
    for item, stars in items:
        filtered_rows.append((user, item, stars))

filtered_df = pd.DataFrame(filtered_rows, columns=["user_num", "item_num", "stars"])

# Step 4: Define replication rule based on stars
def assign_replication(star):
    if star == 5:
        return 3
    elif star >= 3 and star < 5:
        return 2
    else:
        return 1

filtered_df["replication_factor"] = filtered_df["stars"].apply(assign_replication)

# Step 5: Replicate interactions
replicated_rows = []
for row in filtered_df.itertuples():
    replicated_rows.extend([(row.user_num, row.item_num)] * row.replication_factor)

replicated_df = pd.DataFrame(replicated_rows, columns=["user_num", "item_num"])

print("Original interactions:", len(filtered_df))
print("Replicated interactions:", len(replicated_df))
print("Replication distribution:\n", filtered_df["replication_factor"].value_counts())

# Step 6: Split users (70% train, 30% test)
train_users, test_users = train_test_split(list(qualified_users.keys()), train_size=0.7, random_state=42)

train_df = replicated_df[replicated_df["user_num"].isin(train_users)]
test_df = replicated_df[replicated_df["user_num"].isin(test_users)]

# Step 7: Write PAAC format
def write_paac_format(df_subset, filename):
    uc_dict = defaultdict(list)
    for row in df_subset.itertuples():
        uc_dict[row.user_num].append(row.item_num)

    with open(filename, "w") as f:
        for user_id, item_ids in uc_dict.items():
            item_str = " ".join(map(str, sorted(set(item_ids))))
            f.write(f"{user_id} {item_str}\n")

write_paac_format(train_df, "paac1_replicated_train.txt")
write_paac_format(test_df, "paac1_replicated_test.txt")

# Step 8: Save ID mappings
# pd.Series(user_lookup).to_csv("user_lookup.csv", index_label="user_num", header=["user_id"])
# pd.Series(item_lookup).to_csv("item_lookup.csv", index_label="item_num", header=["item_id"])


# qualified users: 25766
Original interactions: 205610
Replicated interactions: 457983
Replication distribution:
 2    95927
3    78223
1    31460
Name: replication_factor, dtype: int64


In [22]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
import random

random.seed(42)

# Copy and parse dates
df = df.copy()
df["date"] = pd.to_datetime(df["date"])

# Encode user and item (business) IDs
df["item_num"], item_lookup = pd.factorize(df["business_id"])
df["user_num"], user_lookup = pd.factorize(df["user_id"])

# Step 1: Build user -> list of (item_num, date)
user_item_dict = defaultdict(list)
for row in df.itertuples():
    user_item_dict[row.user_num].append((row.item_num, row.date))

# Step 2: Filter users with ≥4 unique items
qualified_users = {}
for user, items in user_item_dict.items():
    unique_items = set(item for item, _ in items)
    if len(unique_items) >= 4:
        qualified_users[user] = items

print(f"# qualified users: {len(qualified_users)}")

# Step 3: Flatten interactions (user, item, date)
filtered_rows = []
for user, items in qualified_users.items():
    for item, date in items:
        filtered_rows.append((user, item, date))

filtered_df = pd.DataFrame(filtered_rows, columns=["user_num", "item_num", "date"])

# Step 4: Define replication rule based on recency
# Use quantiles to define "recent"
q_recent = filtered_df["date"].quantile(0.75)
q_mid = filtered_df["date"].quantile(0.50)

def assign_replication(date):
    if date >= q_recent:
        return 3  # most recent
    elif date >= q_mid:
        return 2  # medium recency
    else:
        return 1  # old

filtered_df["replication_factor"] = filtered_df["date"].apply(assign_replication)

# Step 5: Replicate interactions
replicated_rows = []
for row in filtered_df.itertuples():
    replicated_rows.extend([(row.user_num, row.item_num)] * row.replication_factor)

replicated_df = pd.DataFrame(replicated_rows, columns=["user_num", "item_num"])

print("Original interactions:", len(filtered_df))
print("Replicated interactions:", len(replicated_df))
print("Replication by recency:\n", filtered_df["replication_factor"].value_counts())

# Step 6: Split users (70% train, 30% test)
train_users, test_users = train_test_split(list(qualified_users.keys()), train_size=0.7, random_state=42)

train_df = replicated_df[replicated_df["user_num"].isin(train_users)]
test_df = replicated_df[replicated_df["user_num"].isin(test_users)]

# Step 7: Write PAAC format
def write_paac_format(df_subset, filename):
    uc_dict = defaultdict(list)
    for row in df_subset.itertuples():
        uc_dict[row.user_num].append(row.item_num)

    with open(filename, "w") as f:
        for user_id, item_ids in uc_dict.items():
            item_str = " ".join(map(str, sorted(set(item_ids))))
            f.write(f"{user_id} {item_str}\n")

write_paac_format(train_df, "paac_by_recency_train.txt")
write_paac_format(test_df, "paac_by_recency_test.txt")

# Step 8: Save ID mappings
# pd.Series(user_lookup).to_csv("user_lookup.csv", index_label="user_num", header=["user_id"])
# pd.Series(item_lookup).to_csv("item_lookup.csv", index_label="item_num", header=["item_id"])


# qualified users: 25766
Original interactions: 205610
Replicated interactions: 359818
Replication by recency:
 1    102805
3     51403
2     51402
Name: replication_factor, dtype: int64
